In [1]:
import os
import sys
#nb_dir = os.path.split(os.getcwd())[0]
#if nb_dir not in sys.path:
#    sys.path.append(nb_dir)
    
autolib_dir = '/home/kai/data/shiyi/AlphaBoosting/automl'
if autolib_dir not in sys.path:
    sys.path.append(autolib_dir)

print(sys.path)

%load_ext autoreload

%autoreload 2

from automl_libs import feature_engineering as fe
from automl_libs import encoding, kernels, nn_libs 

['', '/home/kai/anaconda3/envs/tf_gpu/lib/python35.zip', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/plat-linux', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/lib-dynload', '/home/kai/.local/lib/python3.5/site-packages', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/IPython/extensions', '/home/kai/.ipython', '/home/kai/data/shiyi/AlphaBoosting/automl']


/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
categorical_features = ['Month', 'DayofMonth', 'DayOfWeek']#, 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance']

In [3]:
import itertools
#params = {'split_col': 't', 'coefficient': 10, 'n': 2, 'fillna': 22}
def get_features_to_gen(function_list):
    features_to_gen = []
    for function in function_list:
        for i in range(2, len(categorical_features)+1):
            for combine in itertools.combinations(categorical_features, i):
                if function.__name__ == 'count_std_over_mean':
                    features_to_gen.append({'params': {'coefficient': 10}, 'function': function, 'feature_cols': list(combine)})
                else:
                    features_to_gen.append({'params': {}, 'function': function, 'feature_cols': list(combine)})
                
    return features_to_gen

#features_to_gen = get_features_to_gen([fe.count, fe.unique_count, fe.cumulative_count,
#                                      fe.reverse_cumulative_count, fe.variance, fe.count_std_over_mean])
features_to_gen = get_features_to_gen([fe.count])

features_to_gen

[{'feature_cols': ['Month', 'DayofMonth'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'DayOfWeek'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['DayofMonth', 'DayOfWeek'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'DayofMonth', 'DayOfWeek'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}}]

In [4]:
len(features_to_gen)

4

In [5]:
project_path = '/home/kai/data/shiyi/AlphaBoosting/automl/automl_app/project1/' 

In [6]:
from automl_app import logger_config

logger_config.config(project_path+'log_project1.log')

In [7]:
def gs_params_gen(model='lgb'):
    import time
    import numpy as np
    seed = int(time.time()* 1000000) % 45234634
    np.random.seed(seed)
    if model == 'lgb':
        params = {    
            'objective': 'binary',
            'boosting': 'gbdt',
            'num_boost_round': 50, # ignored in params. extract it and put it in input arguments in train or cv explicitly
                                   # seems to work fine as the upper limit when combined with early_stopping_round
            'learning_rate': np.random.choice([0.1,0.03]),#0.001]),
            'num_leaves': np.random.choice([15,31]),#,61,127]),
            'num_threads': 8, # best speed: set to number of real cpu cores, which is vCPU/2
            'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
            'min_data_in_leaf': np.random.randint(20,50),  #minimal number of data in one leaf. 
            'feature_fraction': np.random.randint(9,11)/10,
            'feature_fraction_seed': seed,
            'early_stopping_round':5,
            'bagging_freq': 1, #0 means disable bagging. k: perform bagging at every k iteration
            'bagging_fraction': np.random.randint(4,11)/10, #Randomly select part of data 
            'bagging_seed': seed,
            'scale_pos_weight': 1,
            'metric': 'auc' 
        }
    elif model == 'nn':
        params = {
            'nn_seed': int(time.time() * 1000000) % 45234634,
            'ep_for_lr': 1,
            'lr_init': 0.01,
            'lr_fin': 0.005, # if == lr_init, then no lr decay
            'batch_size': np.random.choice([32, 64, 128, 256, 512, 1024, 2048]),
            "pred_batch_size": 100000,
            'max_ep': 1,
            'patience': 5,
            'cat_emb_outdim': 50, # could be a constant or a dict (col name:embed out dim). e.g.:
                                  # embed_outdim = [3, 3, 8, 8, 3]
                                  # embed_outdim_dict = dict(zip(X_train.columns.values, embed_outdim))
            'dense_units': 50,
            'num_dense_n_layers': 1,
            'drop_rate': 0.2,
            'combined_dense_n_layers': 1,
            'monitor': 'val_auc', # or val_loss
            'mode': 'max'
        }
    return params, seed

In [8]:
gs_params_gen('lgb')

({'bagging_fraction': 0.9,
  'bagging_freq': 1,
  'bagging_seed': 14219285,
  'boosting': 'gbdt',
  'early_stopping_round': 5,
  'feature_fraction': 1.0,
  'feature_fraction_seed': 14219285,
  'learning_rate': 0.1,
  'max_depth': -1,
  'metric': 'auc',
  'min_data_in_leaf': 29,
  'num_boost_round': 50,
  'num_leaves': 31,
  'num_threads': 8,
  'objective': 'binary',
  'scale_pos_weight': 1},
 14219285)

In [56]:
from automl_app.app import AlphaBoosting

automl_config_file = project_path + 'automl_config.json'
run_record_file_name = project_path + 'last_run_record.json' # don't created this file
AlphaBoosting(automl_config_file, features_to_gen, gs_params_gen)

2018-06-29 15:32:34 | INFO  | automl_app.app.AlphaBoosting  | __init__   | #64  | Run record file [./output/last_run_record.json] not found. Begin the first time run...
2018-06-29 15:32:34 | INFO  | automl_app.app.AlphaBoosting  | __init__   | #95  | generate todo list
2018-06-29 15:32:34 | INFO  | automl_app.app.AlphaBoosting  | __init__   | #111 | STAGE: FEATURE_ENGINEERING
2018-06-29 15:32:34 | INFO  | automl_app.app.AlphaBoosting  | __init__   | #115 | STAGE: VALIDATION_DOWNSAMPLING_GEN
2018-06-29 15:32:34 | INFO  | automl_app.app.AlphaBoosting  | __init__   | #119 | STAGE: CONCAT_DATA
2018-06-29 15:32:36 | INFO  | automl_app.app.AlphaBoosting  | __init__   | #123 | STAGE: GRID_SEARCH
2018-06-29 15:32:36 | INFO  | automl_app.app.AlphaBoosting  | __init__   | #127 | STAGE: STACKNET
2018-06-29 15:32:37 | DEBUG | automl_libs.stacknet          | layer1     | #37  | data_id: flight_data_ordinal  
	x_train: (250000, 12)	x_test: (100000, 12)
	y_train type: <class 'dict'>
	compatible_model

Computing... label: label        model_data_id: 1530296376__NN_flight_data_ordinal
> /home/kai/data/shiyi/AlphaBoosting/automl/automl_libs/base_layer_utils.py(470)compute_layer1_oof()
-> if isinstance(model, sle.NNBLE):
(Pdb) n
> /home/kai/data/shiyi/AlphaBoosting/automl/automl_libs/base_layer_utils.py(474)compute_layer1_oof()
-> model.train(x_train, y_train)
(Pdb) model
(Pdb) sle.NNBLE
<class 'automl_libs.stack_layer_estimator.NNBLE'>
(Pdb) if isinstance(model, sle.NNBLE)
*** SyntaxError: invalid syntax
(Pdb) isinstance(model, sle.NNBLE)
False
(Pdb) q


BdbQuit: 

In [35]:
class Temp:
    pass

In [36]:
temp = Temp()

In [37]:
isinstance(temp, Temp)

True

In [40]:
from automl_libs import base_layer_utils

In [41]:
temp1 = base_layer_utils.Temp1()

In [42]:
isinstance(temp1, base_layer_utils.Temp1)

True

In [43]:
temp1

In [44]:
base_layer_utils.Temp1

automl_libs.base_layer_utils.Temp1

In [52]:
temp = [1,1,2]

In [53]:
a = [0]

In [54]:
a.extend(temp)

In [55]:
a

[0, 1, 1, 2]